In [1]:
import re
import string

# Simple text feature engineering

In this nitebook we will extract simple features from text field: 
* len of the text
* number of punctuation
* number of digits
* #numbers/len
* number of upper case
* number of spaces

Links: https://www.quora.com/Natural-Language-Processing-What-are-the-possible-features-that-can-be-extracted-from-text

In [2]:
%store -r DATA_NUM_CL_WITH_NAN

In [3]:
data_cl = DATA_NUM_CL_WITH_NAN

In [4]:
data_cl.head()

,url,meta_name,text,tag,color_r,color_g,color_b,font_size,display,font_weight,...,font_family,text_align,line_height,num_siblings,x_coords,y_coords,block_height,block_width,height,width
3,http://www.ticketbis.com/ar/entradas-schalke-0...,name,FC Ingolstadt - Schalke 04,26,94.0,103.0,113.0,11.0,4,400,...,143,5,20.15625,1.0,26.0,1245.0,14.0,163.0,-1.0,-1.0
8,http://www.chicagoparkdistrict.com/parks/Galew...,name,Movie Inside the Park at Mayfair Park,26,51.0,51.0,51.0,26.0,4,300,...,878,7,28.00000,2.0,15.0,200.0,60.0,357.0,-1.0,-1.0
10,http://www.chicagoparkdistrict.com/parks/Galew...,location,Mayfair Park\n\n \n\n 4...,22,0.0,0.0,0.0,13.0,3,400,...,878,7,20.00000,NaN,15.0,313.0,100.0,370.0,100.0,370.0
11,http://www.chicagoparkdistrict.com/parks/Scott...,name,Night Out: Movies in the Parks at Sauganash,26,51.0,51.0,51.0,26.0,4,300,...,878,7,28.00000,2.0,15.0,200.0,60.0,363.0,-1.0,-1.0
14,http://www.chicagoparkdistrict.com/parks/Scott...,description,The Land Before TimeFive orphan dinosaurs trav...,26,0.0,0.0,0.0,13.0,4,400,...,878,7,20.00000,NaN,15.0,493.0,170.0,370.0,-1.0,-1.0


All texts are not null

In [5]:
data_cl.text.isnull().sum()

0

In [6]:
data_cl.shape

(167872, 24)

In [7]:
data_cl['text_str'] = data_cl.text.apply(str).replace('nan','')

In [8]:
data_cl['text_str'] = data_cl['text_str'].apply(lambda x: x.replace('\n', ''))

In [9]:
data_cl['text_len'] = data_cl.text_str.apply(len)

In [10]:
count = lambda l1, l2: len(list(filter(lambda c: c in l2, l1)))

In [11]:
data_cl['num_punctuation'] = data_cl.text_str.apply(lambda x: count(x, string.punctuation))
data_cl['num_digits'] = data_cl.text_str.apply(lambda x: count(x, string.digits))
data_cl['digits_share'] = data_cl.num_digits/data_cl.text_len
data_cl['num_upper'] = data_cl.text_str.apply(lambda x: count(x, string.ascii_uppercase))
data_cl['num_whitespace'] = data_cl.text_str.apply(lambda x: count(x, string.whitespace))

In [12]:
data_cl.head()

,url,meta_name,text,tag,color_r,color_g,color_b,font_size,display,font_weight,...,block_width,height,width,text_str,text_len,num_punctuation,num_digits,digits_share,num_upper,num_whitespace
3,http://www.ticketbis.com/ar/entradas-schalke-0...,name,FC Ingolstadt - Schalke 04,26,94.0,103.0,113.0,11.0,4,400,...,163.0,-1.0,-1.0,FC Ingolstadt - Schalke 04,26,1,2,0.076923,4,4
8,http://www.chicagoparkdistrict.com/parks/Galew...,name,Movie Inside the Park at Mayfair Park,26,51.0,51.0,51.0,26.0,4,300,...,357.0,-1.0,-1.0,Movie Inside the Park at Mayfair Park,37,0,0,0.000000,5,6
10,http://www.chicagoparkdistrict.com/parks/Galew...,location,Mayfair Park\n\n \n\n 4...,22,0.0,0.0,0.0,13.0,3,400,...,370.0,100.0,370.0,Mayfair Park 4550 W. S...,150,6,19,0.126667,7,86
11,http://www.chicagoparkdistrict.com/parks/Scott...,name,Night Out: Movies in the Parks at Sauganash,26,51.0,51.0,51.0,26.0,4,300,...,363.0,-1.0,-1.0,Night Out: Movies in the Parks at Sauganash,43,1,0,0.000000,5,7
14,http://www.chicagoparkdistrict.com/parks/Scott...,description,The Land Before TimeFive orphan dinosaurs trav...,26,0.0,0.0,0.0,13.0,4,400,...,370.0,-1.0,-1.0,The Land Before TimeFive orphan dinosaurs trav...,253,6,6,0.023715,13,41


In [13]:
data_cl['text'] = data_cl['text_str']

In [14]:
del data_cl['text_str']

### Outliers

In [15]:
import numpy as np

In [16]:
def outliers_to_nan(series):
    series_sorted = series.sort_values()
    series_sorted = series_sorted.dropna()
    Q1 = series_sorted.quantile(0.10)
    Q3 = series_sorted.quantile(0.90)
    IQR = Q3 - Q1
    series[series >= Q3 + 1.5 * IQR] = np.nan
    series[series <= Q1 - 1.5 * IQR] = np.nan
    return series

In [17]:
# prepare data for outliers
cols = ['num_punctuation','num_digits','digits_share','num_upper','num_whitespace', 'text_len']

data_out1 = data_cl[cols]

# fill negative values
data_out1[data_out1 < 0] = np.nan

# remove outliers for the first set
data_out1_cl = data_out1.apply(outliers_to_nan)

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)
/Users/jetbrains/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jetbrains

In [18]:
data_cl[cols] = data_out1_cl

In [19]:
DATA_NUM_CL_WITH_NAN_TXT = data_cl

In [20]:
%store DATA_NUM_CL_WITH_NAN_TXT

Stored 'DATA_NUM_CL_WITH_NAN_TXT' (DataFrame)


In [72]:
import h5py
DATA_NUM_CL_WITH_NAN_TXT.to_hdf('store.h5', 'DATA_NUM_CL_WITH_NAN_TXT')